# Introducción

Desarrollo de un modelo de Machine Learning para la identificación automática del hablante.

Se Creó una red Perceptrón Multi Capa, modelando lo planteado en el paper: Identificación automática del hablante mediante redes neuronales de la UNER

Link: https://www.researchgate.net/publication/265964745_Identificacion_Automatica_del_Hablante_mediante_Redes_Neuronales
        
Torres, Humberto & Rufiner, Hugo. (1999). Identificación Automática del Hablante mediante Redes Neuronales.

# 

# 

# Imports

In [189]:
from sklearn.neural_network import MLPClassifier
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
#import tqdm
import librosa
from librosa import display
from joblib import load, dump
from datetime import datetime
import math

# 

# 

# Variables Globales

In [2]:
#Directorio de los archivos de voz TIMIT usado para entrenamiento
directorio_timit_train = "TIMIT USADO\\DR2\\"

#Directorio de los archivos de voz TIMIT usado para entrenamiento
directorio_timit_test = "TIMIT USADO\\TEST\\"

#Cantidad de personas usadas en el entrenamiento
cant_personas_train = 8

#Cantidad de Coeficientes MFCC a extraer(incluidos DELTAS)
cant_MFCC = 32


# 

# 

# CARGA Y ALMACENAMIENTO DE DATOS

Carga de los archivos de audio a utilizar en el entrenamiento del modelo.

Se usó el corpus TIMIT (https://catalog.ldc.upenn.edu/LDC93S1).

Dicho corpus consta de archivos de audio de diferentes hablantes, en inglés, muestreados a 16kHz. Está catalogado en diferentes regiones de dialectos, y también por sexo del hablante.

Los hablantes utilizados son 8 personas de sexo femenino.

En total, se poseen 10 archivos de audio por persona, donde se utilizan 9 para entrenamiento, y el restante para verificación.

En total: 72 archivos de audio para entrenamiento (9 por persona), y 8 archivos de audio para testeo (1 por persona).

Notar cada archivo de audio es el habla de una única persona.

En total, se procesaron 13804 Frames.

# 

# 

# DEFINICIÓN DE FUNCIONES

In [3]:
#Notar que por default, se establecen los valores:

#sr=16000. Tasa de muestreo de los audios de TIMIT

#n_mfcc=16. Cantidad de Coeficientes utilizados en el Paper a replicar

#n_fft=512. Tamaño de la ventana en la transformada de Fourier. (Notar que, sin solapamiento,
#se obtendrán 16000/512 frames por segundo)

#hop_length=256. Tamaño del salto. Con 256, se logra un solapamiento del 0.5 entre cada ventana de la FFT, logrando así
#captar características propias de los pronunciamientos y fonemas.



def loadMfccArchivo(archivoVoz, sr=16000, n_mfcc=16, n_fft=512, hop_length=256):
    """
    Carga de los coeficientes Cepstrum en la escal Mel de un archivo utilizando la librería librosa.
    
    
    
    Params:
    
    archivoVoz = archivo a procesar
    sr = Sampling Rate del archivo (en Hz)
    n_mfcc = cantidad de coeficientes cepstrales a extraer
    n_fft = tamaño de la ventana de la transformada de Fourier
    hop_length = tamaño del salto para el procesamiento en frames
    
    
    
    Para mas información:
    
    MFCC = https://es.wikipedia.org/wiki/MFCC
    
    Librosa MFCC = https://librosa.org/doc/main/generated/librosa.feature.mfcc.html
    """
    y,sr=librosa.load(archivoVoz, sr=sr)
    return librosa.feature.mfcc(y, sr=sr, n_mfcc=n_mfcc, dct_type=2, norm='ortho',
                                hop_length=hop_length, n_fft=n_fft) 

In [4]:
#Funcion para agregar en una matriz, un vector de identificacion.
#El vector identifica, para el i-ésimo vector de la matriz de mfcc, a que persona corresponde dicho MFCC.

#Agrega a la matriz un vector de longitud K, donde K son la cant. personas a identificar, 
#siendo todas las posiciones 0, salvo la persona a la que corresponde el frame que es 1.

#I.E. Siendo K=5 personas a identificar, la persona 3 identificada en el frame, se agregará al vectorIdentificados
#el frame actual, y a personaIdentificada el array (0,0,1,0,0)

#A efectos de replicar el paper, se establece como default la cantidad de personas en 8, en caso de testear con menos
#se deja a libre elección el parámetro


def agregarIdentificacionPersona(personaIdentificada, vectorIdentificados, cantPersonas=8):
    
    """
    Agregado de la identificación de una persona a un vector de personas identificadas, y del frame actual
    a un vector con frames de audios
    
    Agregado en el final, para no perder referencia de que el n-ésimo frame del vector de frames identificados
    corresponde a la persona marcada con 1 en el n-ésimo frame del vector de personas identificadas
    
    Params:
    
    personaIdentificada = número de la persona identificada
    vectorIdentificados = vector de personas identificadas hasta el momento
    cantPersonas = cantidad de personas a analizar pertenencia


    """
    
    #Creo un vector de todos zeros. Luego, con índice comenzando en 0, marco a la persona identificada con 1
    #apilo verticalmente al final, y retorno el vector con la persona nueva identificada
    
    vectorCeros = np.zeros((cantPersonas,))              
    vectorCeros[personaIdentificada-1] = 1
    vectorIdentificados = np.vstack((vectorIdentificados, vectorCeros))
    return vectorIdentificados

# 

# 

# INFORMACIÓN DE SET DE DATOS

Carga de los archivos de audio a utilizar en el entrenamiento del modelo.

Se usó el corpus TIMIT (https://catalog.ldc.upenn.edu/LDC93S1).

Dicho corpus consta de archivos de audio de diferentes hablantes, en inglés, muestreados a 16kHz. Está catalogado en diferentes regiones de dialectos, y también por sexo del hablante.

Los hablantes utilizados son 8 personas de sexo femenino.

En total, se poseen 10 archivos de audio por persona, donde se utilizan 9 para entrenamiento, y el restante para verificación.

En total: 72 archivos de audio para entrenamiento (9 por persona), y 8 archivos de audio para testeo (1 por persona).

Notar cada archivo de audio es el habla de una única persona.

En total, se procesaron 13804 Frames.

# 

# 

In [5]:
#Recorro las carpetas de la región del dialecto, donde cada subcarpeta es una persona.
for persona in sorted(os.listdir(directorio_timit_train)):

    
    #Persona de la que leo los features
    persona_actual += 1

    
    #Contador de frames leidos de los archivos de la persona actual
    cant_frames_persona_actual = 0

    
    #Carpeta de los archivos de audio de cada persona
    for archivo_voz in sorted(os.listdir(directorio_timit_train+persona+"\\")):
        if int(archivo_voz.split('.')[0]) > 4:
            continue
            
        print('persona ',persona, '   archivo ', archivo_voz,'    ',int(archivo_voz.split('.')[0])>4)

NameError: name 'persona_actual' is not defined

# PROCESADO DE AUDIOS DE ENTRENAMIENTO

In [6]:
#Seteo la primer fila de los MFCC, luego la borro 
df_MFCC = np.zeros(cant_MFCC,)


#Seteo la primer fila de personas, luego la borro
df_personas = np.zeros(cant_personas_train,)


#Persona de la que leo los features
persona_actual = 0


#Auxiliar para saber la cantidad de frames totales para train
cant_frames_train = 0


#Recorro las carpetas de la región del dialecto, donde cada subcarpeta es una persona.
for persona in sorted(os.listdir(directorio_timit_train)):

    
    #Persona de la que leo los features
    persona_actual += 1

    
    #Contador de frames leidos de los archivos de la persona actual
    cant_frames_persona_actual = 0

    
    #Carpeta de los archivos de audio de cada persona
    for archivo_voz in sorted(os.listdir(directorio_timit_train+persona+"\\")):
        if int(archivo_voz.split('.')[0]) > 3:
            continue
    
        #Extraigo los MFCC de un audio
        mfcc = loadMfccArchivo(directorio_timit_train+persona+"\\"+archivo_voz)

        
        #Actualmente en el array tengo n filas x m columnas, donde n es la cantidad de MFCC y m la cantidad de frames
        #Traspongo la matriz para quedarme con los features de un determinado frame, es decir n cantidad de MFCC
        #columnas x m cantidad de frames filas
        mfcc = np.transpose(mfcc)

        
        #Para agregar información temporal y fonética (velocidad, aceleración, entre otras) a los coeficientes
        #agrego los delta de cada feature y los concateno al array en formato de columnas
        mfcc = np.append(mfcc, librosa.feature.delta(mfcc), axis = 1)

        
        #apilo los features de los audios a un array
        df_MFCC = np.vstack((df_MFCC, mfcc))

        
        #Cuento los frames de la persona actual
        cant_frames_persona_actual+= mfcc.shape[0]


    #Ya lei todos los archivos de la persona
    #Paso a identificar la persona para almacenar los frames identificados
    for i in range(0, cant_frames_persona_actual):
   
        df_personas = agregarIdentificacionPersona(persona_actual, df_personas)
     
    
    print("Leídos {:d} frames de la persona {:d}".format(cant_frames_persona_actual, persona_actual))
    
    cant_frames_train += cant_frames_persona_actual


#Borro la primer fila de MFCC que eran todos 0
df_MFCC = np.delete(df_MFCC, (0), axis=0)


#Borro la primer fila de Personas que eran todos 0
df_personas = np.delete(df_personas, (0), axis=0)


# #Guardo los MFCC y la identificacion de las personas
# np.savetxt('drive/My Drive/Tesina 2021/Reconocimiento de voz/MfccPersonas.csv', endMfcc, delimiter=',')
# np.savetxt('drive/My Drive/Tesina 2021/Reconocimiento de voz/FramesIdentificados.csv', endPersonas, delimiter=',')




Leídos 542 frames de la persona 1
Leídos 670 frames de la persona 2
Leídos 632 frames de la persona 3
Leídos 572 frames de la persona 4
Leídos 534 frames de la persona 5
Leídos 658 frames de la persona 6
Leídos 589 frames de la persona 7
Leídos 773 frames de la persona 8


In [7]:
cant_frames_train

4970

# 

# 

# PROCESADO DE AUDIOS DE TESTEO

In [8]:
#Seteo la primer fila de los MFCC, luego la borro 
df_MFCC_test = np.zeros(cant_MFCC,)


#Seteo la primer fila de personas, luego la borro
df_personas_test = np.zeros(cant_personas_train,)


#Persona de la que leo los features
persona_actual = 0


#Auxiliar para saber la cantidad de frames totales para train
cant_frames_test = 0


#Recorro las carpetas de la región del dialecto, donde cada subcarpeta es una persona.
for persona in sorted(os.listdir(directorio_timit_test)):

    
    #Persona de la que leo los features
    persona_actual += 1

    
    #Contador de frames leidos de los archivos de la persona actual
    cant_frames_persona_actual = 0

    
    #Carpeta de los archivos de audio de cada persona
    for archivo_voz in sorted(os.listdir(directorio_timit_test+persona+"\\")):
    
    
        #Extraigo los MFCC de un audio
        mfcc = loadMfccArchivo(directorio_timit_test+persona+"\\"+archivo_voz)

        
        #Actualmente en el array tengo n filas x m columnas, donde n es la cantidad de MFCC y m la cantidad de frames
        #Traspongo la matriz para quedarme con los features de un determinado frame, es decir n cantidad de MFCC
        #columnas x m cantidad de frames filas
        mfcc = np.transpose(mfcc)

        
        #Para agregar información temporal y fonética (velocidad, aceleración, entre otras) a los coeficientes
        #agrego los delta de cada feature y los concateno al array en formato de columnas
        mfcc = np.append(mfcc, librosa.feature.delta(mfcc), axis = 1)

        
        #apilo los features de los audios a un array
        df_MFCC_test = np.vstack((df_MFCC_test, mfcc))

        
        #Cuento los frames de la persona actual
        cant_frames_persona_actual+= mfcc.shape[0]


    #Ya lei todos los archivos de la persona
    #Paso a identificar la persona para almacenar los frames identificados
    for i in range(0, cant_frames_persona_actual):
   
        df_personas_test = agregarIdentificacionPersona(persona_actual, df_personas_test)
     
    
    print("Leídos {:d} frames de la persona {:d}".format(cant_frames_persona_actual, persona_actual))
    
    cant_frames_test += cant_frames_persona_actual


#Borro la primer fila de MFCC que eran todos 0
df_MFCC_test = np.delete(df_MFCC_test, (0), axis=0)


#Borro la primer fila de Personas que eran todos 0
df_personas_test = np.delete(df_personas_test, (0), axis=0)



# #Guardo los MFCC y la identificacion de las personas
# np.savetxt('drive/My Drive/Tesina 2021/Reconocimiento de voz/MfccPersonasTEST.csv', endMfcc, delimiter=',')
# np.savetxt('drive/My Drive/Tesina 2021/Reconocimiento de voz/FramesIdentificadosTEST.csv', endPersonas, delimiter=',')



Leídos 1384 frames de la persona 1
Leídos 1375 frames de la persona 2
Leídos 1195 frames de la persona 3
Leídos 1412 frames de la persona 4
Leídos 1082 frames de la persona 5
Leídos 1240 frames de la persona 6
Leídos 1302 frames de la persona 7
Leídos 1602 frames de la persona 8


In [9]:
cant_frames_totales = cant_frames_test+cant_frames_train
print('cant frames test  ',cant_frames_test)
print('cant frames train ',cant_frames_train)
print('cant frames total ',cant_frames_totales)

print('distribucion train {:.2}'.format(cant_frames_train/cant_frames_totales))
print('distribucion test  {:.2}',cant_frames_test/cant_frames_totales)

cant frames test   10592
cant frames train  4970
cant frames total  15562
distribucion train 0.32
distribucion test  {:.2} 0.6806323094717902


# 

# 

# ENTRENAMIENTO DE LA RED

In [10]:
#CREO LA RED CON LAS CARACTERÍSTICAS MENCIONADAS
clasificador_voz = MLPClassifier(hidden_layer_sizes=(75, 75), activation='tanh', solver='adam', alpha=0.0001, 
                            batch_size='auto', learning_rate='adaptive', learning_rate_init=0.001, power_t=0.5, 
                            max_iter=100000, shuffle=True, random_state=None, tol=0.0001, verbose=True, warm_start=True, 
                            momentum=0.9, nesterovs_momentum=True, early_stopping=False, validation_fraction=0.1, beta_1=0.9, 
                            beta_2=0.999, epsilon=1e-08, n_iter_no_change=150, max_fun=15000)



#Entreno la red
clasificador_voz.fit(df_MFCC, df_personas)

Iteration 1, loss = 3.82592919
Iteration 2, loss = 3.02850530
Iteration 3, loss = 2.96043210
Iteration 4, loss = 2.92484506
Iteration 5, loss = 2.88871797
Iteration 6, loss = 2.85583672
Iteration 7, loss = 2.82759519
Iteration 8, loss = 2.79750236
Iteration 9, loss = 2.75577981
Iteration 10, loss = 2.72498783
Iteration 11, loss = 2.69278686
Iteration 12, loss = 2.66238692
Iteration 13, loss = 2.63841053
Iteration 14, loss = 2.61247675
Iteration 15, loss = 2.58877646
Iteration 16, loss = 2.54610988
Iteration 17, loss = 2.52734944
Iteration 18, loss = 2.50822904
Iteration 19, loss = 2.49253452
Iteration 20, loss = 2.47931132
Iteration 21, loss = 2.44976018
Iteration 22, loss = 2.45445724
Iteration 23, loss = 2.44284899
Iteration 24, loss = 2.41913089
Iteration 25, loss = 2.41660781
Iteration 26, loss = 2.39510167
Iteration 27, loss = 2.38180259
Iteration 28, loss = 2.37329843
Iteration 29, loss = 2.36473143
Iteration 30, loss = 2.35044484
Iteration 31, loss = 2.33810132
Iteration 32, los

Iteration 256, loss = 1.73160698
Iteration 257, loss = 1.71911325
Iteration 258, loss = 1.71596777
Iteration 259, loss = 1.70932176
Iteration 260, loss = 1.71602711
Iteration 261, loss = 1.72579755
Iteration 262, loss = 1.72505109
Iteration 263, loss = 1.72354620
Iteration 264, loss = 1.71859850
Iteration 265, loss = 1.70970009
Iteration 266, loss = 1.71672921
Iteration 267, loss = 1.70027044
Iteration 268, loss = 1.71540079
Iteration 269, loss = 1.69895256
Iteration 270, loss = 1.70428124
Iteration 271, loss = 1.70908804
Iteration 272, loss = 1.71420302
Iteration 273, loss = 1.69803510
Iteration 274, loss = 1.70621381
Iteration 275, loss = 1.69193345
Iteration 276, loss = 1.69731166
Iteration 277, loss = 1.70682087
Iteration 278, loss = 1.69046535
Iteration 279, loss = 1.70208668
Iteration 280, loss = 1.69563692
Iteration 281, loss = 1.69912807
Iteration 282, loss = 1.69321971
Iteration 283, loss = 1.68715873
Iteration 284, loss = 1.68097722
Iteration 285, loss = 1.68313389
Iteration 

Iteration 504, loss = 1.51554193
Iteration 505, loss = 1.51295684
Iteration 506, loss = 1.52314068
Iteration 507, loss = 1.52711915
Iteration 508, loss = 1.51256806
Iteration 509, loss = 1.50545056
Iteration 510, loss = 1.50981548
Iteration 511, loss = 1.51906573
Iteration 512, loss = 1.50698495
Iteration 513, loss = 1.50292165
Iteration 514, loss = 1.50266079
Iteration 515, loss = 1.51905692
Iteration 516, loss = 1.50194902
Iteration 517, loss = 1.49510304
Iteration 518, loss = 1.49994227
Iteration 519, loss = 1.50157308
Iteration 520, loss = 1.50855031
Iteration 521, loss = 1.49965302
Iteration 522, loss = 1.49040139
Iteration 523, loss = 1.51085485
Iteration 524, loss = 1.51544087
Iteration 525, loss = 1.50382971
Iteration 526, loss = 1.50049357
Iteration 527, loss = 1.50204923
Iteration 528, loss = 1.49724423
Iteration 529, loss = 1.52342915
Iteration 530, loss = 1.50961228
Iteration 531, loss = 1.53684778
Iteration 532, loss = 1.51178825
Iteration 533, loss = 1.49900669
Iteration 

Iteration 752, loss = 1.39589137
Iteration 753, loss = 1.39795729
Iteration 754, loss = 1.41314140
Iteration 755, loss = 1.41351682
Iteration 756, loss = 1.43301905
Iteration 757, loss = 1.39815672
Iteration 758, loss = 1.39879579
Iteration 759, loss = 1.40583792
Iteration 760, loss = 1.38670377
Iteration 761, loss = 1.39817565
Iteration 762, loss = 1.40638043
Iteration 763, loss = 1.38922050
Iteration 764, loss = 1.39214293
Iteration 765, loss = 1.39387673
Iteration 766, loss = 1.39941634
Iteration 767, loss = 1.38920600
Iteration 768, loss = 1.39270158
Iteration 769, loss = 1.41040513
Iteration 770, loss = 1.39703192
Iteration 771, loss = 1.39166389
Iteration 772, loss = 1.39179622
Iteration 773, loss = 1.39768657
Iteration 774, loss = 1.38878634
Iteration 775, loss = 1.38489408
Iteration 776, loss = 1.38576091
Iteration 777, loss = 1.39229829
Iteration 778, loss = 1.38571223
Iteration 779, loss = 1.38101647
Iteration 780, loss = 1.37885011
Iteration 781, loss = 1.38031000
Iteration 

Iteration 1002, loss = 1.34467542
Iteration 1003, loss = 1.33983953
Iteration 1004, loss = 1.31786453
Iteration 1005, loss = 1.34707471
Iteration 1006, loss = 1.34946130
Iteration 1007, loss = 1.33316152
Iteration 1008, loss = 1.32571882
Iteration 1009, loss = 1.32146397
Iteration 1010, loss = 1.30910259
Iteration 1011, loss = 1.32502961
Iteration 1012, loss = 1.32643144
Iteration 1013, loss = 1.33732840
Iteration 1014, loss = 1.32281848
Iteration 1015, loss = 1.31382510
Iteration 1016, loss = 1.34537884
Iteration 1017, loss = 1.33268955
Iteration 1018, loss = 1.31369436
Iteration 1019, loss = 1.31593833
Iteration 1020, loss = 1.32573149
Iteration 1021, loss = 1.32519312
Iteration 1022, loss = 1.30789092
Iteration 1023, loss = 1.31295303
Iteration 1024, loss = 1.30997604
Iteration 1025, loss = 1.31733508
Iteration 1026, loss = 1.31049425
Iteration 1027, loss = 1.31094900
Iteration 1028, loss = 1.31312518
Iteration 1029, loss = 1.32256200
Iteration 1030, loss = 1.34446504
Iteration 1031

...

Iteration 1726, loss = 1.19574427
Iteration 1727, loss = 1.19813710
Iteration 1728, loss = 1.18639184
Iteration 1729, loss = 1.19161680
Iteration 1730, loss = 1.18490438
Iteration 1731, loss = 1.19826331
Iteration 1732, loss = 1.19137463
Iteration 1733, loss = 1.20961045
Iteration 1734, loss = 1.20819007
Iteration 1735, loss = 1.21662605
Iteration 1736, loss = 1.22131476
Iteration 1737, loss = 1.20016366
Iteration 1738, loss = 1.19089182
Iteration 1739, loss = 1.19152162
Iteration 1740, loss = 1.19842223
Iteration 1741, loss = 1.21075412
Iteration 1742, loss = 1.21277555
Iteration 1743, loss = 1.21269841
Iteration 1744, loss = 1.20824598
Iteration 1745, loss = 1.23589160
Iteration 1746, loss = 1.21365542
Iteration 1747, loss = 1.19797301
Iteration 1748, loss = 1.20244329
Iteration 1749, loss = 1.19176665
Iteration 1750, loss = 1.18085365
Iteration 1751, loss = 1.18401814
Iteration 1752, loss = 1.18852003
Iteration 1753, loss = 1.19693697
Iteration 1754, loss = 1.21178085
Iteration 1755

Iteration 1967, loss = 1.18051548
Iteration 1968, loss = 1.18472545
Iteration 1969, loss = 1.18689743
Iteration 1970, loss = 1.19172809
Iteration 1971, loss = 1.18230851
Iteration 1972, loss = 1.17947663
Iteration 1973, loss = 1.19109177
Iteration 1974, loss = 1.19123000
Iteration 1975, loss = 1.17392227
Iteration 1976, loss = 1.17193154
Iteration 1977, loss = 1.17286420
Iteration 1978, loss = 1.17604279
Iteration 1979, loss = 1.17613427
Iteration 1980, loss = 1.18299728
Iteration 1981, loss = 1.18521486
Iteration 1982, loss = 1.22790632
Iteration 1983, loss = 1.20760789
Iteration 1984, loss = 1.17089807
Iteration 1985, loss = 1.19148716
Iteration 1986, loss = 1.16721280
Iteration 1987, loss = 1.18449123
Iteration 1988, loss = 1.19127852
Iteration 1989, loss = 1.18071224
Iteration 1990, loss = 1.20364226
Iteration 1991, loss = 1.19579025
Iteration 1992, loss = 1.18024497
Iteration 1993, loss = 1.19220519
Iteration 1994, loss = 1.18602083
Iteration 1995, loss = 1.17501213
Iteration 1996

Iteration 2209, loss = 1.16561601
Iteration 2210, loss = 1.16337882
Iteration 2211, loss = 1.15991284
Iteration 2212, loss = 1.16893951
Iteration 2213, loss = 1.14559810
Iteration 2214, loss = 1.14614928
Iteration 2215, loss = 1.16329479
Iteration 2216, loss = 1.15356168
Iteration 2217, loss = 1.14617290
Iteration 2218, loss = 1.15052910
Iteration 2219, loss = 1.16296473
Iteration 2220, loss = 1.16089214
Iteration 2221, loss = 1.16443919
Iteration 2222, loss = 1.18451468
Iteration 2223, loss = 1.20273795
Iteration 2224, loss = 1.21723592
Iteration 2225, loss = 1.20840627
Iteration 2226, loss = 1.20772895
Iteration 2227, loss = 1.20666710
Iteration 2228, loss = 1.16421943
Iteration 2229, loss = 1.16390751
Iteration 2230, loss = 1.17968432
Iteration 2231, loss = 1.17391529
Iteration 2232, loss = 1.15696312
Iteration 2233, loss = 1.16263046
Iteration 2234, loss = 1.16867094
Iteration 2235, loss = 1.17985483
Iteration 2236, loss = 1.16186817
Iteration 2237, loss = 1.15176442
Iteration 2238

Iteration 2451, loss = 1.15058253
Iteration 2452, loss = 1.14298052
Iteration 2453, loss = 1.13690400
Iteration 2454, loss = 1.14514210
Iteration 2455, loss = 1.15101425
Iteration 2456, loss = 1.16174107
Iteration 2457, loss = 1.14829543
Iteration 2458, loss = 1.13967888
Iteration 2459, loss = 1.14442553
Iteration 2460, loss = 1.13601196
Iteration 2461, loss = 1.12373512
Iteration 2462, loss = 1.13043361
Iteration 2463, loss = 1.14200018
Iteration 2464, loss = 1.16092804
Iteration 2465, loss = 1.15726659
Iteration 2466, loss = 1.16091302
Iteration 2467, loss = 1.17147639
Iteration 2468, loss = 1.16749741
Iteration 2469, loss = 1.13674179
Iteration 2470, loss = 1.14867649
Iteration 2471, loss = 1.16069281
Iteration 2472, loss = 1.16401718
Iteration 2473, loss = 1.14689653
Iteration 2474, loss = 1.14453831
Iteration 2475, loss = 1.14973153
Iteration 2476, loss = 1.13842573
Iteration 2477, loss = 1.15406370
Iteration 2478, loss = 1.15802690
Iteration 2479, loss = 1.17392007
Iteration 2480

MLPClassifier(activation='tanh', hidden_layer_sizes=(75, 75),
              learning_rate='adaptive', max_iter=100000, n_iter_no_change=150,
              verbose=True, warm_start=True)

In [26]:
now = datetime.now().strftime('%Y%m%d_%H%M')
dump(clasificador_voz, now+'_RedEntrenada.csv')

['20210906_2004_RedEntrenada.csv']

# 

# 

# TESTEO DE LA RED

In [27]:
#CANTIDAD DE FRAMES MAL CLASIFICADOS
cant_frames_mal_clasificados = 0

#CANTIDAD DE FRAMES BIEN CLASIFICADOS
cant_frames_bien_clasificados = 0



#PARA CADA FRAME DE TESTEO, VEO LA PREDICCIÓN DE LA RED
for i in range(0, df_MFCC_test.shape[0]):
    
    #Si la predicción del i-ésimo frame es igual al i-ésimo arreglo de personas que se identificaron
    #está bien clasificado
    if(np.array_equal(clasificador_voz.predict([df_MFCC_test[i]])[0], df_personas_test[i])):
        cant_frames_bien_clasificados += 1
    else:
        cant_frames_mal_clasificados += 1

        
print("BIEN CLASIFICADOS = ",cant_frames_bien_clasificados)

print("MAL CLASIFICADOS = ",cant_frames_mal_clasificados)


print('score TRAIN:', clasificador_voz.score(df_MFCC, df_personas)) 
print('score TEST:', clasificador_voz.score(df_MFCC_test, df_personas_test)) 

BIEN CLASIFICADOS =  2474
MAL CLASIFICADOS =  8118
score TRAIN: 0.6231388329979879
score TEST: 0.2335725075528701


In [28]:
#CANTIDAD DE FRAMES CLASIFICADOS
cant_frames_mal_clasificados = 0
cant_frames_bien_clasificados = 0

#CANTIDAD DE PERSONAS CLASIFICADAS
cant_personas_mal_clasificadas = 0
cant_personas_bien_clasificadas = 0

frames_persona = np.zeros((8,))


#PARA CADA FRAME DE TESTEO, VEO LA PREDICCIÓN DE LA RED
for i in range(0, df_MFCC_test.shape[0]):
    
    
    #Si la predicción del i-ésimo frame es igual al i-ésimo arreglo de personas que se identificaron
    #está bien clasificado  
    
    if(np.array_equal(clasificador_voz.predict([df_MFCC_test[i]])[0], df_personas_test[i])):
        cant_frames_bien_clasificados += 1
        
    else:
        cant_frames_mal_clasificados += 1


        
#VEO LA PREDICCIÓN POR PERSONAS DE LA RED
for persona in range(0, cant_personas_train):

    #RECORRO LOS FRAMES DE TESTEO PARA VALIDAR LA RED
    for frame in range(0, df_MFCC_test.shape[0]):
  
        #ME QUEDO CON LOS FRAMES QUE IDENTIFIQUE QUE SEAN DE LA PERSONA
        #ESTOS SON LOS FRAMES IDENTIFICADOS QUE USO PARA TESTEO (VALIDACION)
        if(df_personas_test[frame][persona] == 1):
      
            #RECORRO DE 0 a 7, 8 PERSONAS. PARA SABER SI EL OUTPUT QUE ME DIO LA RED
            #ES EL DE LA PERSONA QUE YO NECESITO
            for j in range(0, cant_personas_train):
        
                #SI ES EL DE ESA PERSONA, SETEO QUE LE IDENTIFIQUE UN FRAME
                if(clasificador_voz.predict([df_MFCC_test[frame]])[0][j] == 1):
                    frames_persona[j] += 1
  
    #YA RECORRI TODOS LOS FRAMES DE TESTEO, AHORA ME FIJO CON QUE PERSONA SE
    #IDENTIFICARON LA MAYOR CANTIDAD DE FRAMES

    persona_mas_frames = 0
    cant_frames_mayor_de_persona_identificada = 0

    #VEO EL VECTOR DONDE ME GUARDE LA CANTIDAD DE FRAMES QUE IDENTIFIQUE PARA
    #CADA PERSONA
    for persona_identificacion in range(0,cant_personas_train):

        if(frames_persona[persona_identificacion] > cant_frames_mayor_de_persona_identificada):
            cant_frames_mayor_de_persona_identificada = frames_persona[persona_identificacion]
            persona_mas_frames = persona_identificacion

    #YA TENGO QUE PERSONA FUE CON LA QUE MAS FRAMES SE IDENTIFICARON VEO SI ES LA
    #QUE TENIA QUE SER

    if(persona_mas_frames == persona):
        cant_personas_bien_clasificadas += 1
    else:
        cant_personas_mal_clasificadas += 1



    print("ERA LA PERSONA ",persona," IDENTIFIQUE :",frames_persona)
    frames_persona = np.zeros((8,))




print("FRAMES TEST BIEN CLASIFICADOS = ",cant_frames_bien_clasificados)
print("FRAMES TEST MAL CLASIFICADOS = ",cant_frames_mal_clasificados)
print("PERSONAS TEST BIEN CLASIFICADAS = ",cant_personas_bien_clasificadas)
print("PERSONAS TEST MAL CLASIFICADAS = ",cant_personas_mal_clasificadas)


print('score FRAMES TRAIN:', clasificador_voz.score(df_MFCC, df_personas)) 
print('score FRAMES TEST:', clasificador_voz.score(df_MFCC_test, df_personas_test)) 
print('score PERSONAS TEST:', cant_personas_train/cant_personas_bien_clasificadas)

ERA LA PERSONA  0  IDENTIFIQUE : [425. 105.  39.  42.  62.  79.  74.  87.]
ERA LA PERSONA  1  IDENTIFIQUE : [ 71. 378.  89.  67. 103.  93. 112.  68.]
ERA LA PERSONA  2  IDENTIFIQUE : [ 31. 143. 299.  64.  61.  53.  34.  84.]
ERA LA PERSONA  3  IDENTIFIQUE : [ 54.  46.  77. 344.  82.  87.  75. 153.]
ERA LA PERSONA  4  IDENTIFIQUE : [ 59. 105.  55.  63. 233.  53.  72.  86.]
ERA LA PERSONA  5  IDENTIFIQUE : [ 51.  79.  31.  92.  37. 384.  77.  81.]
ERA LA PERSONA  6  IDENTIFIQUE : [ 65. 104.  49.  36.  68.  61. 336. 115.]
ERA LA PERSONA  7  IDENTIFIQUE : [ 51.  78. 162.  78.  74.  67. 111. 609.]
FRAMES TEST BIEN CLASIFICADOS =  2474
FRAMES TEST MAL CLASIFICADOS =  8118
PERSONAS TEST BIEN CLASIFICADAS =  8
PERSONAS TEST MAL CLASIFICADAS =  0
score FRAMES TRAIN: 0.6231388329979879
score FRAMES TEST: 0.2335725075528701
score PERSONAS TEST: 1.0


In [36]:
cant_frames_totales = cant_frames_test+cant_frames_train
print('cant frames test  ',cant_frames_test)
print('cant frames train ',cant_frames_train)
print('cant frames total ',cant_frames_totales)

print('distribucion train {:.2%}'.format(cant_frames_train/cant_frames_totales))
print('distribucion test  {:.2%}'.format(cant_frames_test/cant_frames_totales))

cant frames test   10592
cant frames train  4970
cant frames total  15562
distribucion train 31.94%
distribucion test  68.06%


In [101]:
#Testeo de cada audio que deje para test



#Seteo la primer fila de los MFCC, luego la borro 
df_MFCC_test = np.zeros(cant_MFCC,)


#Seteo la primer fila de personas, luego la borro
df_personas_test = np.zeros(cant_personas_train,)


#Persona de la que leo los features
persona_actual = 0


#Auxiliar para saber la cantidad de frames totales para train
cant_frames_test = 0


#CANTIDAD TOTAL DE FRAMES CLASIFICADOS
cant_total_frames_mal_clasificados = 0
cant_total_frames_bien_clasificados = 0

#CANTIDAD DE AUDIOS CLASIFICADOS
cant_audios_mal_clasificados = 0
cant_audios_bien_clasificados = 0


#Recorro las carpetas de la región del dialecto, donde cada subcarpeta es una persona.
for persona in sorted(os.listdir(directorio_timit_test)):

    
    #Persona de la que leo los features
    persona_actual += 1

    
    #Contador de frames leidos de los archivos de la persona actual
    cant_frames_persona_actual = 0

    
    #Carpeta de los archivos de audio de cada persona
    for archivo_voz in sorted(os.listdir(directorio_timit_test+persona+"\\")):
    
        
        #Extraigo los MFCC de un audio
        mfcc = loadMfccArchivo(directorio_timit_test+persona+"\\"+archivo_voz)

        
        #Actualmente en el array tengo n filas x m columnas, donde n es la cantidad de MFCC y m la cantidad de frames
        #Traspongo la matriz para quedarme con los features de un determinado frame, es decir n cantidad de MFCC
        #columnas x m cantidad de frames filas
        mfcc = np.transpose(mfcc)
        
        
        #Para agregar información temporal y fonética (velocidad, aceleración, entre otras) a los coeficientes
        #agrego los delta de cada feature y los concateno al array en formato de columnas
        mfcc = np.append(mfcc, librosa.feature.delta(mfcc), axis = 1)
        
        
        #Contador de frames leidos de los archivos de la persona actual
        cant_frames_audio_actual = mfcc.shape[0]
        
        print("Leídos {:d} frames del audio {:s} de la persona {:d}".format(cant_frames_audio_actual, 
                                                                            archivo_voz,
                                                                            persona_actual))
        
    
        
        #CANTIDAD DE FRAMES CLASIFICADOS DEL AUDIO ACTUAL
        cant_frames_audio_actual_mal_clasificados = 0
        cant_frames_audio_actual_bien_clasificados = 0
        
        
        #ARRAY PARA ASIGNAR LAS PREDICCIONES DE CADA FRAME DEL AUDIO ACTUAL, A PERSONAS
        predicciones_frames_personas = np.zeros((cant_personas_train,))

        
        #Recorro todos los frames del audio
        for frame in mfcc:
            
            prediccion = clasificador_voz.predict([frame])[0]
            
            if prediccion[persona_actual-1] == 1:
                cant_frames_audio_actual_bien_clasificados+=1
            else:
                cant_frames_audio_actual_mal_clasificados+=1
         
            #RECORRO EL FRAME PARA SABER A QUE PERSONA CORRESPONDE LA PREDICCION
            for i in range(0, prediccion.shape[0]):
        
                #SI ES EL DE ESA PERSONA, SETEO QUE LE IDENTIFIQUE UN FRAME, Y NO RECORRO MÁS
                if(prediccion[i] == 1):
                    predicciones_frames_personas[i] += 1
                    break
            
        
        #PERSONA IDENTIFICADA. INDICE MAXIMO +1 
        persona_identificada = predicciones_frames_personas.argmax()+1
        
        
        if(persona_identificada == persona_actual):
            cant_audios_bien_clasificados += 1
        else:
            cant_audios_mal_clasificados += 1

        
        
        cant_total_frames_mal_clasificados += cant_frames_audio_actual_mal_clasificados
        cant_total_frames_bien_clasificados += cant_frames_audio_actual_bien_clasificados
        
        
        print("ERA LA PERSONA {:s}, LA IDENTIFIQUE COMO LA {:d}. IDENTIFICACION FRAMES : "
              .format(persona,persona_identificada)
              ,predicciones_frames_personas)        
        
        
        
        
#         break
#     break



Leídos 281 frames del audio 4.WAV de la persona 1
ERA LA PERSONA P1, LA IDENTIFIQUE COMO LA 1. IDENTIFICACION FRAMES :  [70. 19.  8.  5. 12. 17. 12. 15.]
Leídos 145 frames del audio 5.WAV de la persona 1
ERA LA PERSONA P1, LA IDENTIFIQUE COMO LA 1. IDENTIFICACION FRAMES :  [48. 12.  5.  3.  3.  5.  1.  4.]
Leídos 143 frames del audio 6.WAV de la persona 1
ERA LA PERSONA P1, LA IDENTIFIQUE COMO LA 1. IDENTIFICACION FRAMES :  [47. 10.  2.  5.  3.  3. 11.  4.]
Leídos 186 frames del audio 7.WAV de la persona 1
ERA LA PERSONA P1, LA IDENTIFIQUE COMO LA 1. IDENTIFICACION FRAMES :  [73.  7.  2.  1.  7.  5.  9.  3.]
Leídos 273 frames del audio 8.WAV de la persona 1
ERA LA PERSONA P1, LA IDENTIFIQUE COMO LA 1. IDENTIFICACION FRAMES :  [88. 25.  7.  8.  1. 10.  9.  6.]
Leídos 143 frames del audio 9.WAV de la persona 1
ERA LA PERSONA P1, LA IDENTIFIQUE COMO LA 1. IDENTIFICACION FRAMES :  [41.  3.  1.  4.  8. 11.  9. 10.]
Leídos 213 frames del audio SX295.WAV de la persona 1
ERA LA PERSONA P1, LA 

ERA LA PERSONA P8, LA IDENTIFIQUE COMO LA 8. IDENTIFICACION FRAMES :  [  7.  25.  34.  13.  12.  17.  16. 128.]


In [102]:
print("Audios BIEN clasificados = ",cant_audios_bien_clasificados)
print("Audios MAL clasificados  = ",cant_audios_mal_clasificados)
print("Frames BIEN clasificados = ",cant_total_frames_bien_clasificados)
print("Frames MAL clasificados  = ",cant_total_frames_mal_clasificados)

Audios BIEN clasificados =  53
Audios MAL clasificados  =  3
Frames BIEN clasificados =  3008
Frames MAL clasificados  =  7584


In [200]:
#Testeo de cada audio que deje para test


filas_dataframe = [[None,None,None,None]]

#Persona de la que leo los features
persona_actual = 0


#Auxiliar para saber la cantidad de frames totales para train
cant_frames_test = 0


#CANTIDAD TOTAL DE FRAMES CLASIFICADOS
cant_total_frames_mal_clasificados = 0
cant_total_frames_bien_clasificados = 0

#CANTIDAD DE AUDIOS CLASIFICADOS
cant_audios_mal_clasificados = 0
cant_audios_bien_clasificados = 0


#Recorro las carpetas de la región del dialecto, donde cada subcarpeta es una persona.
for persona in sorted(os.listdir(directorio_timit_test)):

    
    #Persona de la que leo los features
    persona_actual += 1

    
    #Contador de frames leidos de los archivos de la persona actual
    cant_frames_persona_actual = 0

    
    #Carpeta de los archivos de audio de cada persona
    for archivo_voz in sorted(os.listdir(directorio_timit_test+persona+"\\")):
    
        
        #Extraigo los MFCC de un audio
        mfcc = loadMfccArchivo(directorio_timit_test+persona+"\\"+archivo_voz)

        
        #Actualmente en el array tengo n filas x m columnas, donde n es la cantidad de MFCC y m la cantidad de frames
        #Traspongo la matriz para quedarme con los features de un determinado frame, es decir n cantidad de MFCC
        #columnas x m cantidad de frames filas
        mfcc = np.transpose(mfcc)
        
        
        #Para agregar información temporal y fonética (velocidad, aceleración, entre otras) a los coeficientes
        #agrego los delta de cada feature y los concateno al array en formato de columnas
        mfcc = np.append(mfcc, librosa.feature.delta(mfcc), axis = 1)
        
        
        
        
        
        
        
        
        #Contador de frames leidos de los archivos de la persona actual
        cant_frames_audio_actual = mfcc.shape[0]
        
#         print("Leídos {:d} frames del audio {:s} de la persona {:d}".format(cant_frames_audio_actual, 
#                                                                             archivo_voz,
#                                                                             persona_actual))
        
    
        
        #CANTIDAD DE FRAMES CLASIFICADOS DEL AUDIO ACTUAL
        cant_frames_audio_actual_mal_clasificados = 0
        cant_frames_audio_actual_bien_clasificados = 0
        
        
        #ARRAY PARA ASIGNAR LAS PREDICCIONES DE CADA FRAME DEL AUDIO ACTUAL, A PERSONAS
        predicciones_frames_personas = np.zeros((cant_personas_train,))

        
        #Recorro todos los frames del audio
        for frame in mfcc:
            
            prediccion = clasificador_voz.predict([frame])[0]
            
            if prediccion[persona_actual-1] == 1:
                cant_frames_audio_actual_bien_clasificados+=1
            else:
                cant_frames_audio_actual_mal_clasificados+=1
         
            #RECORRO EL FRAME PARA SABER A QUE PERSONA CORRESPONDE LA PREDICCION
            for i in range(0, prediccion.shape[0]):
        
                #SI ES EL DE ESA PERSONA, SETEO QUE LE IDENTIFIQUE UN FRAME, Y NO RECORRO MÁS
                if(prediccion[i] == 1):
                    predicciones_frames_personas[i] += 1
                    break
            
            
        filas_dataframe.append([archivo_voz, persona_actual, cant_frames_audio_actual, predicciones_frames_personas])
#         #PERSONA IDENTIFICADA. INDICE MAXIMO +1 
#         persona_identificada = predicciones_frames_personas.argmax()+1
        
        
#         if(persona_identificada == persona_actual):
#             cant_audios_bien_clasificados += 1
#         else:
#             cant_audios_mal_clasificados += 1

        
        
#         cant_total_frames_mal_clasificados += cant_frames_audio_actual_mal_clasificados
#         cant_total_frames_bien_clasificados += cant_frames_audio_actual_bien_clasificados
        
        
#         print("ERA LA PERSONA {:s}, LA IDENTIFIQUE COMO LA {:d}. IDENTIFICACION FRAMES : "
#               .format(persona,persona_identificada)
#               ,predicciones_frames_personas)        
        
        
        
        
#         break
#     break



In [222]:
columns = ['Audio', 'Persona del audio', 'Cantidad de frames', 'Predicciones en frames']


df_resultado = pd.DataFrame(filas_dataframe, columns=columns)


df_resultado.drop(0, inplace=True)


df_resultado['Persona del audio'] = df_resultado['Persona del audio'].astype(int)


df_resultado['Frames bien clasificados'] = df_resultado.apply(lambda row : 
                                                              row['Predicciones en frames'][row['Persona del audio']-1]
                                                              .astype(int), axis=1)


df_resultado['Frames mal clasificados'] = df_resultado.apply(lambda row : 
                                                              (row['Predicciones en frames'].sum() -
                                                               row['Frames bien clasificados'])
                                                              .astype(int), axis=1)


df_resultado['Frames no clasificados'] = df_resultado.apply(lambda row : 
                                                              (row['Cantidad de frames'] - 
                                                               row['Predicciones en frames'].sum())
                                                              .astype(int), axis=1)


df_resultado['Porcentaje no clasificados'] = df_resultado.apply(lambda row : 
                                                              math.ceil(row['Frames no clasificados'] / 
                                                               row['Cantidad de frames']*100)
                                                              , axis=1)


df_resultado['Persona predecida'] = df_resultado['Predicciones en frames'].apply(lambda x : x.argmax()+1)


df_resultado['Prediccion correcta'] = df_resultado['Persona predecida'] == df_resultado['Persona del audio']


df_resultado['Persona segunda max frames'] = (df_resultado['Predicciones en frames']
                                              .apply(lambda x : np.argwhere(x==np.sort(x)[-2])[0][0]+1))


df_resultado['Frames persona segunda max frames'] = df_resultado.apply(lambda row : 
                                                            (row['Predicciones en frames'][row['Persona segunda max frames']-1])
                                                              .astype(int)
                                                              , axis=1)


df_resultado['Porcentaje diferencia primera persona con segunda'] = (df_resultado['Frames bien clasificados'] /
                                                                    df_resultado['Frames persona segunda max frames']*100)

df_resultado


,Audio,Persona del audio,Cantidad de frames,Predicciones en frames,Frames bien clasificados,Frames mal clasificados,Frames no clasificados,Porcentaje no clasificados,Persona predecida,Prediccion correcta,Persona segunda max frames,Frames persona segunda max frames,Porcentaje diferencia primera persona con segunda
1,4.WAV,1,281.0,"[70.0, 19.0, 8.0, 5.0, 12.0, 17.0, 12.0, 15.0]",70,88,123,44,1,True,2,19,368.421053
2,5.WAV,1,145.0,"[48.0, 12.0, 5.0, 3.0, 3.0, 5.0, 1.0, 4.0]",48,33,64,45,1,True,2,12,400.000000
3,6.WAV,1,143.0,"[47.0, 10.0, 2.0, 5.0, 3.0, 3.0, 11.0, 4.0]",47,38,58,41,1,True,7,11,427.272727
4,7.WAV,1,186.0,"[73.0, 7.0, 2.0, 1.0, 7.0, 5.0, 9.0, 3.0]",73,34,79,43,1,True,7,9,811.111111
5,8.WAV,1,273.0,"[88.0, 25.0, 7.0, 8.0, 1.0, 10.0, 9.0, 6.0]",88,66,119,44,1,True,2,25,352.000000
6,9.WAV,1,143.0,"[41.0, 3.0, 1.0, 4.0, 8.0, 11.0, 9.0, 10.0]",41,46,56,40,1,True,6,11,372.727273
7,SX295.WAV,1,213.0,"[58.0, 18.0, 5.0, 4.0, 8.0, 12.0, 1.0, 15.0]",58,63,92,44,1,True,2,18,322.222222
8,4.WAV,2,245.0,"[9.0, 70.0, 17.0, 5.0, 15.0, 15.0, 24.0, 8.0]",70,93,82,34,2,True,7,24,291.666667
9,5.WAV,2,147.0,"[9.0, 34.0, 7.0, 8.0, 12.0, 7.0, 8.0, 5.0]",34,56,57,39,2,True,5,12,283.333333
10,6.WAV,2,201.0,"[9.0, 43.0, 9.0, 10.0, 17.0, 10.0, 16.0, 6.0]",43,77,81,41,2,True,5,17,252.941176


True     53
False     3
Name: Prediccion correcta, dtype: int64

In [137]:
data = [[None,None,None,None,None,None,None,None]]
columns = ['Audio', 'Persona del audio', 'Cantidad de frames', 'Predicciones en frames', 'Frames bien clasificados', 
           'Frames mal clasificados', 'Persona predecida','Prediccion correcta']
#data.append(['a1.WAV', 1, 84])
#data.append(['a1.WAV', 1, 84])


df_resultado = pd.DataFrame(data, columns=[columns])

#df_resultado.set_index('Audio', inplace=True)

#dict = {'Audio': 'a1.WAV', 'Persona del audio': 1, 'Cantidad de frames': 84}

#df_resultado.append({'Audio': 'a1.WAV', 'Persona del audio': 1, 'Cantidad de frames': 84}, ignore_index = True)
#df_resultado.append(['a1.WAV', 1, 84, [70,10,10,15,50,55,10,1], 70, 180, 1, 1], ignore_index = True)

df_resultado


,Audio,Persona del audio,Cantidad de frames,Predicciones en frames,Frames bien clasificados,Frames mal clasificados,Persona predecida,Prediccion correcta
0,None,None,None,None,None,None,None,None
